# Exemple avec des modèles N-grammes de mots
## Survol des principales fonctionnalités de NLTK.
## Découpage des textes en sous-séquences de longueur N, c.-à-d. en N-grammes.

In [20]:
from nltk import word_tokenize, bigrams, trigrams

text = "Le cours IFT-7022 est offert à distance cette année."
tokens = word_tokenize(text)
print(list(bigrams(tokens)))

[('Le', 'cours'), ('cours', 'IFT-7022'), ('IFT-7022', 'est'), ('est', 'offert'), ('offert', 'à'), ('à', 'distance'), ('distance', 'cette'), ('cette', 'année'), ('année', '.')]


In [21]:
print(list(trigrams(tokens))) 


[('Le', 'cours', 'IFT-7022'), ('cours', 'IFT-7022', 'est'), ('IFT-7022', 'est', 'offert'), ('est', 'offert', 'à'), ('offert', 'à', 'distance'), ('à', 'distance', 'cette'), ('distance', 'cette', 'année'), ('cette', 'année', '.')]


## Ajout de symboles de début et de fin de séquence


In [22]:
from nltk.util import pad_sequence 

BOS = '<BOS>'
EOS = '<EOS>'

text = "Le cours IFT-7022 est offert à distance cette année."
tokens = word_tokenize(text)
list(pad_sequence(tokens, pad_left=True, left_pad_symbol=BOS, pad_right=True, right_pad_symbol=EOS, n=2))

['<BOS>',
 'Le',
 'cours',
 'IFT-7022',
 'est',
 'offert',
 'à',
 'distance',
 'cette',
 'année',
 '.',
 '<EOS>']

## Les étapes suivantes donnnent un exemple simple de construction de modèle N-gramme.
Les étapes sont:

* la construction du vocabulaire
* la construction des n-grammes
* l'entraînement du modèle n-gramme
* l'utilisation du modèle pour estimer des probabilités ou des perplexités

In [23]:
from nltk import word_tokenize

def build_vocabulary(text_list):
    all_unigrams = list()
    for sentence in text_list:
        word_list = word_tokenize(sentence.lower())
        all_unigrams = all_unigrams + word_list
    voc = set(all_unigrams)
    voc.add(BOS)
    return list(voc)

corpus = ["le cours ift 7022 est offert à distance cette année .", 
          "ce cours n est habituellement pas à distance .",
          "le cours est habituellement donnée à l automne ."]

vocabulary = build_vocabulary(corpus)
print(vocabulary) 

['<BOS>', '.', 'à', 'ift', 'cours', 'habituellement', 'automne', 'ce', 'offert', 'cette', 'année', 'pas', 'donnée', 'le', 'l', '7022', 'est', 'n', 'distance']


In [24]:
from nltk.util import ngrams

def get_ngrams(text_list, n=2):
    all_ngrams = list()
    for sentence in text_list:
        tokens = word_tokenize(sentence.lower())
        padded_sent = list(pad_sequence(tokens, pad_left=True, left_pad_symbol=BOS, pad_right=True, right_pad_symbol=EOS, n=n))
        all_ngrams = all_ngrams + list(ngrams(padded_sent, n=n))      
    return all_ngrams

order = 2

corpus_ngrams = get_ngrams(corpus, n=order)
display(corpus_ngrams)

[('<BOS>', 'le'),
 ('le', 'cours'),
 ('cours', 'ift'),
 ('ift', '7022'),
 ('7022', 'est'),
 ('est', 'offert'),
 ('offert', 'à'),
 ('à', 'distance'),
 ('distance', 'cette'),
 ('cette', 'année'),
 ('année', '.'),
 ('.', '<EOS>'),
 ('<BOS>', 'ce'),
 ('ce', 'cours'),
 ('cours', 'n'),
 ('n', 'est'),
 ('est', 'habituellement'),
 ('habituellement', 'pas'),
 ('pas', 'à'),
 ('à', 'distance'),
 ('distance', '.'),
 ('.', '<EOS>'),
 ('<BOS>', 'le'),
 ('le', 'cours'),
 ('cours', 'est'),
 ('est', 'habituellement'),
 ('habituellement', 'donnée'),
 ('donnée', 'à'),
 ('à', 'l'),
 ('l', 'automne'),
 ('automne', '.'),
 ('.', '<EOS>')]

In [25]:
from nltk.lm import MLE

model = MLE(order)
model.fit([corpus_ngrams], vocabulary_text=vocabulary)
len(model.vocab)

20

In [26]:
model.score("ift", ["cours"])


0.3333333333333333

In [27]:
model.logscore("ift", ["cours"])


-1.5849625007211563

In [28]:
test_sequence = [("le", "cours"), ("cours", "ift")]
model.perplexity(test_sequence)

1.7320508075688774

In [29]:
model.generate(text_seed=['cours'])


'est'

## Lissage de probabilités
Il faut prévoir que le modèle n'aura pas tout vu à l'entraînement. Et qu'il ne sera pas en mesure d'évaluer de nouvelles séquences de mots.

In [30]:
test_sequence = [("le", "cours"), ("cours", "ift"), ("ift", "est"), ("est", "ce"), ("ce", "soir"), ("soir", ".")]
print("Probabilité de \"ce soir\" = ", model.score("soir", ["ce"]))
print("Perplexité de la séquence de test = ", model.perplexity(test_sequence))

Probabilité de "ce soir" =  0.0
Perplexité de la séquence de test =  inf


On doit alors attribuer des probabilités aux n-grammes inconnus à l'aide d'une technique de lissage. Ici, on applique un lissage de Laplace pour corriger notre problème. D'autres types de lissage sont disponibles dans NLTK.

In [31]:
from nltk.lm.models import Laplace

model = Laplace(order)
model.fit([corpus_ngrams], vocabulary_text=vocabulary)
print("Probabilité de \"ce soir\" = ", model.score("soir", ["ce"]))
print("Perplexité de la séquence de test = ", model.perplexity(test_sequence))

Probabilité de "ce soir" =  0.047619047619047616
Perplexité de la séquence de test =  16.052128016712338
